実習12-1 形態素解析と文の数値化
---
自然言語処理を行う上で、ニューラルネットワークやその他アルゴリズムで計算を行うためには、言語を数値化する必要がある。

今回は、自然言語処理の最初のステップである分かち書き、形態素解析、数値化を中心に、前処理の一部を確認する。


# 1. 形態素解析

形態素解析とは、自然言語処理（NLP）の一種で、文章を意味を持つ最小単位である形態素に分割し、品詞を判別する処理である。

自然言語処理では、最初のステップとして文章を形態素に分割する必要がある。また、用途に応じて、特定の品詞のみを抽出することもできる。

## 1-1. 英語の形態素解析

まずは、英語の形態素解析を行ってみる。英語の場合は、単語間がスペースで区切られており、比較的簡単に行うことが出来る。今回はPythonのライブラリであるNLTK（Natural Language Tool Kit）を使用する。

### 1-1-1. ライブラリの使用準備（NLTK）
Google Colabにはnltkはインストール済のため、importすることはできるが、必要なリソースはnltk.download('リソース名')でダウンロードする必要がある。

```
import nltk
nltk.download('punkt_tab') # 分かち書きに必要
nltk.download('averaged_perceptron_tagger_eng') # 品詞の取得に必要
```

In [ ]:
# NLTKの準備
# 未インストールならpip install nltkなどでインストール
import nltk
nltk.download('punkt_tab') # 分かち書きに必要
nltk.download('averaged_perceptron_tagger_eng') # 品詞の取得に必要

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

### 1-1-2. 分かち書きをする（NLTK）
単語ごとに分割してリストにする。
色々な文を分かち書きしてみよう。
```
nltk.word_tokenize("This is a pen.")  # 直接入れてもいい
string = "I am 20 years old."
nltk.word_tokenize(str) # 文字列変数を入れてもいい
# 後のことを考えて変数に入れておこう
```

In [ ]:
# 分かち書きをして、表示して確認する。
string = "The early bird catches the worm."
words =  nltk.word_tokenize(string)
words
#-> ['The', 'early', 'bird', 'catches', 'the', 'worm', '.']

['The', 'early', 'bird', 'catches', 'the', 'worm', '.']

### 1-1-3. 品詞の取得（NLTK）
上で分かち書きしたものを、nltk.pos_tag()の引数に入れると、
単語と品詞をタプルにしたもののリストを返す。
```
nltk.pos_tag(分かち書きしたもの)
```

In [ ]:
# 品詞の取得
string = "The early bird catches the worm."
words =  nltk.word_tokenize(string)
nltk.pos_tag(words)

[('The', 'DT'),
 ('early', 'JJ'),
 ('bird', 'NN'),
 ('catches', 'VBZ'),
 ('the', 'DT'),
 ('worm', 'NN'),
 ('.', '.')]


[('The', 'DT'),
 ('early', 'JJ'),
 ('bird', 'NN'),
 ('catches', 'VBZ'),
 ('the', 'DT'),
 ('worm', 'NN'),
 ('.', '.')]

※ 出力は、単語と品詞のタプルのリストになっています。特定の品詞の単語のみ取り出す場合の参考にしてください。

**[課題] 品詞の略称（NNなど）について、分かる範囲でいいので調べてください。**

*   例）NNS：名詞（複数形）
*   
*   

## 1-2. 日本語の形態素解析（MeCab）
次に、日本語の形態素解析を行ってみる。日本語は単語間がスペースで区切られておらず、自力で分かち書きを行うことが難しい。

日本語の形態素解析ツールとしては、MeCab、juman、Janomeなどが有名である。
まずはMeCabを使用してみる。

### 1-2-1. ライブラリの使用準備（Mecab）
Google ColabにはMeCabがインストールされていないようなので、まずはインストールする。
MeCabの本体（mecab-python3）と辞書（unidic-lite）をpipでインストールする。
```
# 本体と辞書をインストール
!pip install mecab-python3 unidic-lite

```

インストール後、importすると使用できる。
```
# 後のコードで一緒に実行する
import MeCab
```

In [ ]:
# 本体と辞書をインストール
pip install mecab-python3 unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 5.9 MB/s eta 0:00:00
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=106e63749b6102a9feb6094e68ba36e7dc102fb06dc1b336ba0c18fca4fc9128
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic-lite


### 1-2-2. 分かち書きをする（MeCab）
分かち書きをするために、MeCab.Taggerインスタンスを作成する。作成したインスタンスのparse()メソッドに分かち書きしたい文を入れる。
分かち書きのみを実施する場合は、インスタンス作成時の引数に"-Owakati"を指定する。

```
import MeCab
text = "すもももももももものうち"
m = MeCab.Tagger("-Owakati")
print(m.parse(text))

```

In [ ]:
# MeCabを使用した分かち書き
import MeCab
text = "すもももももももものうち"
m = MeCab.Tagger("-Owakati")
print(m.parse(text))

すもも も もも も もも の うち 



※ 結果がひとつの文字列として出力されています。

### 1-2-3. 品詞情報の取得（MeCab）
MeCab.Taggerは標準で品詞情報なども出力することができる。引数を指定せずにインスタンスを作成して、parseを実行してみる。
```
# インスタンス作成（引数指定なし）
m = MeCab.Tagger()
# 形態素解析の例
text = "日本語の形態素解析は難しかったが少し分かった。"
print(m.parse(text))
```

In [ ]:
# インスタンス作成（引数指定なし）
m = MeCab.Tagger()
# 形態素解析の例
text = "日本語の形態素解析は難しかったが少し分かった。"
print(m.parse(text))

日本	ニッポン	ニッポン	日本	名詞-固有名詞-地名-国			3
語	ゴ	ゴ	語	名詞-普通名詞-一般			1
の	ノ	ノ	の	助詞-格助詞			
形態	ケータイ	ケイタイ	形態	名詞-普通名詞-一般			0
素	ソ	ソ	素	接尾辞-名詞的-一般			
解析	カイセキ	カイセキ	解析	名詞-普通名詞-サ変可能			0
は	ワ	ハ	は	助詞-係助詞			
難しかっ	ムズカシカッ	ムズカシイ	難しい	形容詞-一般	形容詞	連用形-促音便	0,4
た	タ	タ	た	助動詞	助動詞-タ	終止形-一般	
が	ガ	ガ	が	助詞-接続助詞			
少し	スコシ	スコシ	少し	副詞			2
分かっ	ワカッ	ワカル	分かる	動詞-一般	五段-ラ行	連用形-促音便	2
た	タ	タ	た	助動詞	助動詞-タ	終止形-一般	
。			。	補助記号-句点			
EOS



※ 結果がひとつの文字列として出力されています。

## 1-3. 日本語の形態素解析（Janome）
次に、Janomeを使用して日本語の分かち書きと形態素解析を実施してみる。MeCabの場合とはほぼ同じだが、出力形式などが異なるため注意する。

### 1-3-1. ライブラリの使用準備（Janome）
Google ColabにはJanomeがインストールされていないようなので、まずはpipでインストールする。
```
# Janomeをインストール
!pip install janome

```

In [ ]:
# Janomeをインストール
!pip install janome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 61.3 MB/s eta 0:00:00


### 1-3-2. 分かち書きをする（Janome）

分かち書きをするために、 janome.tokenizer.Tokenizer()インスタンスを作成する。作成したインスタンスのtokenize()メソッドに分かち書きしたい文を入れる。分かち書きのみを実施したい場合は、インスタンス作成時またはtokenize時のオプションで「wakati=True」を指定する。

ただし、**分かち書きした結果は文字列ではなくオブジェクトであるため、for文を使って出力する。**

```
# 書き方は様々あり
from janome.tokenizer import Tokenizer
text = "すもももももももものうち"
j = Tokenizer(wakati=True)
for token in j.tokenize(text):
  print(token)
```

In [ ]:
# 書き方は様々あり
from janome.tokenizer import Tokenizer
text = "すもももももももものうち"
j = Tokenizer(wakati=True)
for token in j.tokenize(text):
  print(token)

すもも
も
もも
も
もも
の
うち


※ 各tokenは文字列で出力されています。

### 1-3-3. 品詞情報の取得（Janome）
 janome.tokenizer.Tokenizer()は標準で品詞情報なども出力することができる。引数を指定せずにインスタンスを作成して、tokenizeを実行してみる。


In [ ]:
# 書き方は様々あり
from janome.tokenizer import Tokenizer
text = "すもももももももものうち"
j = Tokenizer()
for token in j.tokenize(text):
  print(token)

すもも	名詞,一般,*,*,*,*,すもも,スモモ,スモモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
うち	名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ


※ 出力は、janome.tokenizer.Tokenクラスであり、各要素には、以下のようにアクセスできる。

- `surface`: 表層形（元の文章に出現する単語そのもの）
- `part_of_speech`: 品詞情報（名詞、動詞、形容詞など）
- `infl_type`: 活用型（動詞や形容詞の場合）
- `infl_form`: 活用形（動詞や形容詞の場合）
- `base_form`: 基本形（辞書形）
- `reading`: 読み（カタカナ）
- `phonetic`: 発音（カタカナ）

```
# 各トークンの詳細情報（一部）
result = j.tokenize("日本語の形態素解析は難しかったが少し分かった。")
for token in result:
    print(f"表層形: {token.surface}")
    print(f"品詞: {token.part_of_speech}")
    print(f"基本形: {token.base_form}")
    print(f"読み: {token.reading}")
    print("---")
```

In [ ]:
# 各トークンの詳細情報（一部）
result = j.tokenize("日本語の形態素解析は難しかったが少し分かった。")
for token in result:
    print(f"表層形: {token.surface}")
    print(f"品詞: {token.part_of_speech}")
    print(f"基本形: {token.base_form}")
    print(f"読み: {token.reading}")
    print("---")

表層形: 日本語
品詞: 名詞,一般,*,*
基本形: 日本語
読み: ニホンゴ
---
表層形: の
品詞: 助詞,連体化,*,*
基本形: の
読み: ノ
---
表層形: 形態素
品詞: 名詞,一般,*,*
基本形: 形態素
読み: ケイタイソ
---
表層形: 解析
品詞: 名詞,サ変接続,*,*
基本形: 解析
読み: カイセキ
---
表層形: は
品詞: 助詞,係助詞,*,*
基本形: は
読み: ハ
---
表層形: 難しかっ
品詞: 形容詞,自立,*,*
基本形: 難しい
読み: ムズカシカッ
---
表層形: た
品詞: 助動詞,*,*,*
基本形: た
読み: タ
---
表層形: が
品詞: 助詞,接続助詞,*,*
基本形: が
読み: ガ
---
表層形: 少し
品詞: 副詞,助詞類接続,*,*
基本形: 少し
読み: スコシ
---
表層形: 分かっ
品詞: 動詞,自立,*,*
基本形: 分かる
読み: ワカッ
---
表層形: た
品詞: 助動詞,*,*,*
基本形: た
読み: タ
---
表層形: 。
品詞: 記号,句点,*,*
基本形: 。
読み: 。
---


# 2. 文章の数値化
文章を機械学習やディープラーニングに入力する場合は、基本的に以下の処理を実施する。
- 分かち書き<br/>
  **※ 今回はすべて品詞の単語を使用するものとする。**
- 各要素を数値に変換
- 長さを一定にする（パディングまたはカット）

今回は、Pythonのライブラリである[tensorflow.keras.preprocessing.text.Tokenizer](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer)を使用して、ここまでの処理を実施する。

## 2-1. 英文の数値化

### 2-1-1. 分かち書き（英語）

英文の場合は、単語間がスペースで区切られており、はじめから分かち書きができている状態である。また、[tensorflow.keras.preprocessing.text.Tokenizer](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer) は、単語がスペース区切りになっている文章が前提であるため、英文では特に必要ない。

ここでは、使用する文章のみを設定しておく。
```
# 使用する文章
sentences_e = [
    'I love my dog.',
    'I love my cat.',
    'You love my dog. I am happy.'
]
```

In [ ]:
# 使用する文章
sentences_e = [
    'I love my dog.',
    'I love my cat.',
    'You love my dog. I am happy.'
]

### 2-1-2. 単語リストの作成（英語）
単語に数値を割り当てるため、
複数の文章を入力として、
単語と数値（ID）のリストを作成する。

　まず、Tokenizerのインスタンスを作成する。インスタンス作成時に、保存する単語数（```num_words```）や、辞書にない単語の取り扱い（```oov_token```）などを指定できる。
　次に、Tokenizerの```fit_on_text```に文章を入力し、各単語にインデックス（ID）を割り当てる。

```
# ライブラリのインポート
from tensorflow.keras.preprocessing.text import Tokenizer
# Tokenizerの作成（保存する単語数10、oovを<OOV>に設定）
tokenizer_e = Tokenizer(num_words = 10, oov_token='<OOV>')

# 各単語にインデックスを割り当てる
tokenizer_e.fit_on_texts(sentences_e)

# 単語リストを表示してみる
word_index_e = tokenizer_e.word_index
print(word_index_e)
```


In [ ]:
# ライブラリのインポート
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenizerの作成（保存する単語数10、oovを<OOV>に設定）
tokenizer_e = Tokenizer(num_words = 10, oov_token='<OOV>')

# 各単語にインデックスを割り当てる
tokenizer_e.fit_on_texts(sentences_e)

# 単語リストを表示してみる
word_index_e = tokenizer_e.word_index
word_index_e

{'<OOV>': 1,
 '最近だと、生成aiであいさつの例文がたくさん出てくるので、無難なものは誰でも作成できる。': 2,
 '求人案内の上から順に履歴書を送り、最初に内定した会社に入社しました。': 3,
 'あいさつがしっかりできる人は、会社でも信頼されやすいです。': 4}

### 2-1-3. 文を数値リストに変換（英語）
単語と数値の対応表ができたので、文を数値のリストに変換することができる。また、入力文の長さを揃えるため、maxlenを超える単語数の場合はカットし、足りない場合はパディングを実施する。

```
# 数値のリストに変換（上と同じsentencesを入れる場合）
sequences_e = tokenizer_e.texts_to_sequences(sentences_e)

# パラメータを指定してpadding（最後をカットまたは最後にpadding。長さを10にそろえる。）
from tensorflow.keras.preprocessing.sequence import pad_sequences
sequences_padded_e = pad_sequences(sequences_e, maxlen=10, padding='post', truncating='post')

# 表示して確認
print("変換前：", sentences_e)
print("変換表：", word_index_e)
print("変換後：" , sequences_e)
print("padding後：", sequences_padded_e)
```

In [ ]:
# 数値のリストに変換（上と同じsentencesを入れる場合）
sequences_e = tokenizer_e.texts_to_sequences(sentences_e)

# パラメータを指定してpadding（最後をカットまたは最後にpadding。長さを10にそろえる。）
from tensorflow.keras.preprocessing.sequence import pad_sequences
sequences_padded_e = pad_sequences(sequences_e, maxlen=10, padding='post', truncating='post')

# 表示して確認
print("変換前：", sentences_e)
print("変換表：", word_index_e)
print("変換後：" , sequences_e)
print("padding後：", sequences_padded_e)

変換前： ['I love my dog.', 'I love my cat.', 'You love my dog. I am happy.']
変換表： {'<OOV>': 1, 'i': 2, 'love': 3, 'my': 4, 'dog': 5, 'cat': 6, 'you': 7, 'am': 8, 'happy': 9}
変換後： [[2, 3, 4, 5], [2, 3, 4, 6], [7, 3, 4, 5, 2, 8, 9]]
padding後： [[2 3 4 5 0 0 0 0 0 0]
 [2 3 4 6 0 0 0 0 0 0]
 [7 3 4 5 2 8 9 0 0 0]]


## 2-2. 和文の数値化（課題）
日本語の場合も、英語の場合とほぼ同じ手順で数値リストに変換することができる。ただし、単語間がスペースで区切られていないため、分かち書きは別途実施する必要がある。

### 2-2-1. 分かち書き（日本語）
MeCabまたはJanomeを使用して分かち書きをする。

```
# 元の文のリスト
sentence = [
    '最近だと、生成AIであいさつの例文がたくさん出てくるので、無難なものは誰でも作成できる。',
    '求人案内の上から順に履歴書を送り、最初に内定した会社に入社しました。',
    'あいさつがしっかりできる人は、会社でも信頼されやすいです。'
]

# MeCabまたはJanomeを使用して、分かち書き後の文（sentence_j）を作成する。

```

In [52]:
# 元の文のリスト
sentence = [
    '最近だと、生成AIであいさつの例文がたくさん出てくるので、無難なものは誰でも作成できる。',
    '求人案内の上から順に履歴書を送り、最初に内定した会社に入社しました。',
    'あいさつがしっかりできる人は、会社でも信頼されやすいです。'
]

In [53]:
# MeCabまたはJanomeを使用して、分かち書き後の文（sentence_j）を作成する。
from janome.tokenizer import Tokenizer

tokenizer = Tokenizer()
sentence_j = []
for text in sentence:
    tokens = [token.surface for token in tokenizer.tokenize(text)]
    sentence_j.append(" ".join(tokens))

print(sentence_j)

['最近 だ と 、 生成 AI で あいさつ の 例文 が たくさん 出 て くる ので 、 無難 な もの は 誰 でも 作成 できる 。', '求人 案内 の 上 から 順に 履歴 書 を 送り 、 最初 に 内定 し た 会社 に 入社 し まし た 。', 'あいさつ が しっかり できる 人 は 、 会社 で も 信頼 さ れ やすい です 。']


### 2-2-2. 単語リストの作成（日本語）
分かち書きができれば、以降は英文と手順は同じである。
Tokenizerを作成し、fit_on_textsを使用して各単語にインデックスを割り当てる。
**※保存する単語数は20ぐらいがいいかもしれない**

In [58]:
# ライブラリのインポート
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenizerの作成（保存する単語数10、oovを<OOV>に設定）
tokenizer_e = Tokenizer(num_words = 20, oov_token='<OOV>')

# 各単語にインデックスを割り当てる
tokenizer_e.fit_on_texts(sentence_j)

# 単語リストを表示してみる
word_index_e = tokenizer_e.word_index
word_index_e

{'<OOV>': 1,
 '、': 2,
 '。': 3,
 'で': 4,
 'あいさつ': 5,
 'の': 6,
 'が': 7,
 'は': 8,
 'できる': 9,
 'に': 10,
 'し': 11,
 'た': 12,
 '会社': 13,
 '最近': 14,
 'だ': 15,
 'と': 16,
 '生成': 17,
 'ai': 18,
 '例文': 19,
 'たくさん': 20,
 '出': 21,
 'て': 22,
 'くる': 23,
 'ので': 24,
 '無難': 25,
 'な': 26,
 'もの': 27,
 '誰': 28,
 'でも': 29,
 '作成': 30,
 '求人': 31,
 '案内': 32,
 '上': 33,
 'から': 34,
 '順に': 35,
 '履歴': 36,
 '書': 37,
 'を': 38,
 '送り': 39,
 '最初': 40,
 '内定': 41,
 '入社': 42,
 'まし': 43,
 'しっかり': 44,
 '人': 45,
 'も': 46,
 '信頼': 47,
 'さ': 48,
 'れ': 49,
 'やすい': 50,
 'です': 51}

### 2-2-3. 文を数値リストに変換（日本語）
英文と同じように実施し、以下のような結果（maxlen=30の場合）を出力する。

```
padding後： [[14 15 16  2 17 18  4  5  6 19  7 20 21 22 23 24  2 25 26 27  8 28 29 30  9  3  0  0  0  0]
 [31 32  6 33 34 35 36 37 38 39  2 40 10 41 11 12 13 10 42 11 43 12  3  0  0  0  0  0  0  0]
 [ 5  7 44  9 45  8  2 13  4 46 47 48 49 50 51  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
   ```

In [59]:
# 数値のリストに変換（上と同じsentencesを入れる場合）
sequences_e = tokenizer_e.texts_to_sequences(sentence_j)

# padding
sequences_padded_e = pad_sequences(sequences_e, maxlen=30, padding='post', truncating='post')

# 結果顯示
print("padding後：", sequences_padded_e)

padding後： [[14 15 16  2 17 18  4  5  6 19  7  1  1  1  1  1  2  1  1  1  8  1  1  1
   9  3  0  0  0  0]
 [ 1  1  6  1  1  1  1  1  1  1  2  1 10  1 11 12 13 10  1 11  1 12  3  0
   0  0  0  0  0  0]
 [ 5  7  1  9  1  8  2 13  4  1  1  1  1  1  1  3  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]]


**pad_sequences()のパラメータを変更したり、Tokenizerのnum_wordsを小さくしたりして動作を確認し、以下を調べてみましょう。**

**[課題]**
*   max_lengthより短い文の場合、padding（長さを同じにするための穴埋め）に使用される数値は何ですか。**[　0　]**
*   変換表にない単語を変換した場合、出力の数値は何になりますか。**[　1　]**



# 提出について

以下を確認してもらうこと。

*   和文を数値リストに変換した結果
*   テキストブロックへの追記（padding、辞書にない単語に使用される数値）

ファイルが保存されているかを確認し、「ファイル＞ダウンロード＞.ipynbをダウンロード」を順にクリックして.ipynbファイルをダウンロードする。

ダウンロードしたipynbファイルを指定の場所に提出してください。